In [1]:
import pickle, random, copy, platform, os
import numpy as np
import tensorflow as tf
import email_sending
from PIL import Image, ImageDraw

In [2]:
def vessel_img():
    with open('vessel', 'rb') as vessel:
        data = pickle.load(vessel)
        band1 = data[0]
        band2 = data[1]

    return band1, band2

In [3]:
#band1
band1 = vessel_img()[0]
band2 = vessel_img()[1]

In [4]:
X1 = band1.reshape(-1, 5625)
X2 = band2.reshape(-1, 5625)

In [ ]:
'''X1 = np.array([X1[0] for i in range(len(X1))])
X2 = np.array([X2[0] for i in range(len(X1))])'''

In [5]:
# GAN Modeling

import matplotlib.pyplot as plt

total_epoch = 10000
batch_size = 37 #851=37*23
learning_rate = 0.00002
# layer option
n_noise = 1024 # the size of noise 
n_input = 75*75
n_hidden =2024

In [6]:
# X is the input, we do not use the labels Y of images
X = tf.placeholder(tf.float32, [None, n_input])
# Z is the noise 
Z = tf.placeholder(tf.float32, [None, n_noise])

In [ ]:
g_W1 = tf.Variable(tf.random_normal([n_noise, n_hidden], mean=0, stddev=0.01, dtype=tf.float32))
g_b1 = tf.Variable(tf.random_normal([n_hidden],stddev=0.01))
g_W2 = tf.Variable(tf.random_normal([n_hidden, n_input], mean=0, stddev=0.01, dtype=tf.float32))
g_b2 = tf.Variable(tf.random_normal([n_input],stddev=0.01))

In [ ]:
def generator(noise_z):
    hidden = tf.nn.relu(tf.matmul(noise_z, g_W1) + g_b1)
    output = tf.nn.sigmoid(tf.matmul(hidden, g_W2) + g_b2)
    Gen_IMG = tf.reshape(output, [-1, 75, 75, 1])
    return Gen_IMG

In [ ]:
d_W1 = tf.Variable(tf.random_normal([3, 3, 1, 32], stddev=0.01))
d_W2 = tf.Variable(tf.random_normal([3, 3, 32, 64], stddev=0.01))
d_W3 = tf.get_variable("d_W3", shape=[64*19*19, n_noise], initializer=tf.contrib.layers.xavier_initializer())
d_b1 = tf.Variable(tf.random_normal([n_noise]))
d_W4 = tf.get_variable("d_W4", shape=[n_noise, 1], initializer=tf.contrib.layers.xavier_initializer())
d_b2 = tf.Variable(tf.random_normal([1]))

In [ ]:
def discriminator(inputs):
    X_img = tf.reshape(inputs, [-1, 75, 75, 1]) 
    
    #Layer1
    L1 = tf.nn.conv2d(X_img, d_W1, strides=[1, 1, 1, 1], padding='SAME')
    L1 = tf.nn.relu(L1)
    L1 = tf.nn.max_pool(L1, ksize=[1, 2, 2, 1],strides=[1, 2, 2, 1], padding='SAME')
    #Layer2
    L2 = tf.nn.conv2d(L1, d_W2, strides=[1, 1, 1, 1], padding='SAME')
    L2 = tf.nn.relu(L2)
    L2 = tf.nn.max_pool(L2, ksize=[1, 2, 2, 1],strides=[1, 2, 2, 1], padding='SAME')
    L2_flat = tf.reshape(L2, [-1, 64 * 19 * 19 ])
    
   # L3 FC 64*19*19 inputs -> 1024 outputs
    L3 = tf.nn.relu(tf.matmul(L2_flat, d_W3) + d_b1)
    #Final FC 1024 inputs -> 1 outputs
    logits = tf.matmul(L3, d_W4) + d_b2
    output = tf.nn.sigmoid(logits)
    return output

In [ ]:
def get_noise(batch_size, n_noise):
    return np.random.normal(size=(batch_size, n_noise))

In [ ]:
# Generate the random image
G = generator(Z)
# Discriminate the generated image
D_generate = discriminator(G)
# Discriminate the real image
D_real = discriminator(X)

In [ ]:
loss_D_real = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(
                        logits=D_real, labels=tf.ones_like(D_real)))

In [ ]:
loss_D_generate = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(
    logits=D_generate, labels=tf.zeros_like(D_generate)))

In [ ]:
loss_D = loss_D_real + loss_D_generate

In [ ]:
loss_G = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_generate, labels=tf.ones_like(D_generate)))

In [ ]:
d_var_list = [d_W1, d_b1, d_W2, d_b2, d_W3, d_W4]
g_var_list = [g_W1, g_b1, g_W2, g_b2]

In [ ]:
train_D = tf.train.AdamOptimizer(learning_rate).minimize(loss_D,var_list=d_var_list)
train_G = tf.train.AdamOptimizer(learning_rate).minimize(loss_G,var_list=g_var_list)

In [ ]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

total_batch = int(851/batch_size)
loss_val_D, loss_val_G = 0, 0

In [ ]:
for epoch in range(total_epoch):
    for i in range(total_batch):
        batch_xs= X1[i*batch_size:(i+1)*batch_size]
        noise = get_noise(batch_size, n_noise)

        # learn the discriminator and generator.
        _, loss_val_D = sess.run([train_D, loss_D],
                                 feed_dict={X: batch_xs, Z: noise})
        _, loss_val_G = sess.run([train_G, loss_G],
                                 feed_dict={Z: noise})

    print('Epoch:', '%04d' % epoch,
          'D loss: {:.4}'.format(loss_val_D),
          'G loss: {:.4}'.format(loss_val_G))
    
    #########
    # save the images which is generated
    ######
    if epoch == 0 or (epoch + 1) % 5 == 0:
        sample_size = 10
        noise = get_noise(sample_size, n_noise)
        samples = sess.run(G, feed_dict={Z: noise})

        fig, ax = plt.subplots(1, sample_size, figsize=(sample_size, 1))

        for i in range(sample_size):
            ax[i].set_axis_off()
            ax[i].imshow(np.reshape(samples[i], (75, 75)), cmap=plt.cm.bone, interpolation='nearest')

        plt.savefig('samples/{}.png'.format(str(epoch).zfill(4)), bbox_inches='tight')
        plt.close(fig)
        print(samples[0])
print('최적화 완료!')